In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
from unidecode import unidecode

# pd.set_option('display.max_columns', None)

## Départements

In [ ]:
t1_dep = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/bf616688-8478-46fa-8abe-cd1cc1a53b83')

t1_dep.drop(t1_dep.columns[16:40], axis=1, inplace=True)
t1_dep.drop(t1_dep.columns[[3, 5, 6, 7, 8, 10, 11, 13, 14]], axis=1, inplace=True)

t1_dep.columns = ['Code Département','Département', 'Inscrits', 'Taux Abstention', 'Taux Blanc', 'Taux Nul', 'Taux Exprimé',
                  'ARTHAUD', 'ROUSSEL', 'MACRON', 'LASSALLE', 'LE PEN', 'ZEMMOUR', 'MÉLENCHON', 'HIDALGO', 'JADOT', 'PÉCRESSE', 'POUTOU', 'DUPONT-AIGNAN']

In [ ]:
t1_dep.head()

In [ ]:
geo_dep = gpd.read_file('https://www.data.gouv.fr/fr/datasets/r/92f37c92-3aae-452c-8af1-c77e6dd590e5')

for i in geo_dep.index[96:]:
    dep = geo_dep.loc[i, 'libgeo']
    code = geo_dep.loc[i, 'dep']
    
    t1_dep.loc[t1_dep['Département'] == dep, 'Code Département'] = code

In [ ]:
t1_dep.head()

## Communes

In [ ]:
t1_com = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/38107a2b-67e6-4d74-a179-c587a25dc6f7',
                     low_memory=False)

t1_com.drop(t1_com.columns[17:41], axis=1, inplace=True)
t1_com.drop(t1_com.columns[[4, 6, 7, 8, 9, 11, 12, 14, 15]], axis=1, inplace=True)

t1_com.columns = ['Code Commune', 'Code Département','Commune', 'Inscrits', 'Taux Abstention', 'Taux Blanc', 'Taux Nul', 'Taux Exprimé',
                  'ARTHAUD', 'ROUSSEL', 'MACRON', 'LASSALLE', 'LE PEN', 'ZEMMOUR', 'MÉLENCHON', 'HIDALGO', 'JADOT', 'PÉCRESSE', 'POUTOU', 'DUPONT-AIGNAN']

In [ ]:
t1_com.head()

In [ ]:
geo_com = gpd.read_file('https://www.data.gouv.fr/fr/datasets/r/fb3580f6-e875-408d-809a-ad22fc418581')

t1_com['Code Département'].replace(['ZA', 'ZB', 'ZC', 'ZD', 'ZM'], 
                                   ['971', '972', '973', '974', '976'], inplace=True)

com_indices = geo_com[(geo_com['dep'] == '2A')
                      | (geo_com['dep'] == '2B')
                      | (geo_com['dep'] > '95')].index

for i in com_indices:
    com = geo_com.loc[i, 'libgeo']
    code_com = geo_com.loc[i, 'codgeo']
    com_dep = geo_com.loc[i, 'dep']
    
    t1_com.loc[(t1_com['Commune'].apply(unidecode) == unidecode(com))
               & (t1_com['Code Département'] == com_dep), ['Code Commune', 'Commune']] = [code_com, com]

In [ ]:
geo_com.head()

## Cartes

In [ ]:
fig = px.choropleth_mapbox(t1_dep,
                           locations='Code Département',
                           geojson=geo_dep,
                           color='Taux Abstention',
                           color_continuous_scale='delta',
                           range_color=[0, 100],
                           featureidkey='properties.dep',
                           hover_name='Département',
                           mapbox_style='white-bg',
                           center={'lat': 46.3, 'lon': 1.6751},
                           zoom=4)
fig.update_layout(margin={'l': 0, 'r': 0, 't': 0, 'b': 0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(t1_com,
                           locations='Code Commune',
                           geojson=geo_com,
                           color='Taux Abstention',
                           color_continuous_scale='delta',
                           range_color=[0, 100],
                           featureidkey='properties.codgeo',
                           hover_name='Commune',
                           mapbox_style='white-bg',
                           center={'lat': 46.3, 'lon': 1.6751},
                           zoom=4)
fig.update_traces(marker_line_width=0)
fig.update_layout(margin={'l': 0, 'r': 0, 't': 0, 'b': 0})
fig.show()